In [13]:
import pandas as pd

df = pd.read_pickle("reviews_segment.pkl")
df.head()

,review_id,product_id,customer_id,review_title,review_written_date,customer_name,review_from_title,review_text,helpful_count,out_of_helpful_count,customer_review_rating,number_of_comments,amazon_verified_purchase,amazon_vine_program_review,review_with_metadata
0,'R10019MUX6F9A','B00006881R','AWNC1GQ75W8K8','Works as advertised','2002-12-17','Neil','TeleZapper TZ 900 (Office Product)','I\'ve had this product for about a month and ...,7,7,4,0,0,0,"('R10019MUX6F9A', 'B00006881R', 'AWNC1GQ75W8K8..."
1,'R1002I943QCT20','B00471F0NK','A3SFG0OC59UXL5',"'macintosh version - bad graphics, bad interface'",'2007-06-05','D. Simons','null','I have been using the Macintosh OSX version o...,21,23,2,0,0,0,"('R1002I943QCT20', 'B00471F0NK', 'A3SFG0OC59UX..."
2,'R1003RILN06MX1','B0027U258Q','A2IP26LJGTJXSV','Great Software','2010-12-05','Tex','Paragon Partition Manager 10 Personal Edition...,'The Partition Manager is a great product. It\...,1,1,5,0,1,0,"('R1003RILN06MX1', 'B0027U258Q', 'A2IP26LJGTJX..."
3,'R100523NBIQIEV','B000070MRB','A2DKAPBHZ5DERR','Neutral','2004-06-07','S. Barnes','Game Programming Starter Kit 6.0 (CD-ROM)','If you plan on getting this program go to htt...,-1,-1,3,0,0,0,"('R100523NBIQIEV', 'B000070MRB', 'A2DKAPBHZ5DE..."
4,'R1006KJEGKGV0O','B001B19D7I','AMZ7EO048MCWK',"'Great seat, but don\'t like the buckle'",'2009-07-07','Cyrca','Britax Boulevard 65 TSIP Convertible Car Seat...,'I researched for months (on-line and in store...,1,1,5,0,0,0,"('R1006KJEGKGV0O', 'B001B19D7I', 'AMZ7EO048MCW..."


In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)